In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
from sklearn.model_selection import train_test_split
import lightgbm as lgbm

In [3]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [4]:
train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train['feature_stock_id_sum'] = train['feature_41'] + train['feature_42'] + train['feature_43']
train['feature_1_2_cross'] = train['feature_1']/(train['feature_2']+1e-5)
train.fillna(-9999,inplace=True)


features = [c for c in train.columns if "feature" in c]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [5]:
valid_df = train.loc[(train.date >= 450) & (train.date < 500)].reset_index(drop=True)
train_df= train.loc[train.date < 450].reset_index(drop=True)
print(train_df.shape)
print(valid_df.shape)

(1335359, 140)
(236056, 140)


In [6]:
X_train = train_df[features].values

y_train= np.stack([(train_df[c] > 0).astype('int') for c in resp_cols]).T

X_valid = valid_df[features].values

y_valid= np.stack([(valid_df[c] > 0).astype('int') for c in resp_cols]).T

In [7]:
import os, sys, gc, time, warnings, pickle, psutil, random
params={"num_leaves":300, # 300-500
        "feature_fraction":0.52, #0.5-0.8
        "bagging_fraction":0.52, #0.5-0.8
        "objective":"binary",
        "learning_rate":0.05, # 0.001-0.01
        "boosting_type":"gbdt",
        "metric":"auc",
        'device':'gpu',
        'seed':xx
    
       }
models = [] 
    
for i in tqdm(range(y_train.shape[1])):
    d_train = lgbm.Dataset(X_train,label=y_train[:,i])
    d_eval = lgbm.Dataset(X_valid,label=y_valid[:,i],reference=d_train)
    clf = lgbm.train(params,d_train,valid_sets=[d_train,d_eval],num_boost_round=1500,\
                    early_stopping_rounds=50,verbose_eval=50)
    models.append(clf)
    model_name = 'lgb_model_'+str(i)+'.bin'
    pickle.dump(clf, open(model_name, 'wb'))

  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 672284, number of negative: 663075
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 33408
[LightGBM] [Info] Number of data points in the train set: 1335359, number of used features: 132
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 132 dense feature groups (168.10 MB) transferred to GPU in 0.196455 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503448 -> initscore=0.013793
[LightGBM] [Info] Start training from score 0.013793
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.621464	valid_1's auc: 0.581489
[100]	training's auc: 0.649934	valid_1's auc: 0.584025
[150]	training's auc: 0.672457	valid_1's auc: 0.584912
[200]	training's auc: 0.691887	valid_1'

 20%|██        | 1/5 [01:50<07:22, 110.57s/it]

[LightGBM] [Info] Number of positive: 675016, number of negative: 660343
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 33408
[LightGBM] [Info] Number of data points in the train set: 1335359, number of used features: 132
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 132 dense feature groups (168.10 MB) transferred to GPU in 0.204958 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.505494 -> initscore=0.021977
[LightGBM] [Info] Start training from score 0.021977
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.617183	valid_1's auc: 0.566249
[100]	training's auc: 0.6504	valid_1's auc: 0.566979
[150]	training's auc: 0.674821	valid_1's auc: 0.567207
Early stopping, best iteration is:
[130]	

 40%|████      | 2/5 [03:20<04:55, 98.41s/it] 

...

 60%|██████    | 3/5 [04:20<02:41, 80.67s/it]

[LightGBM] [Info] Number of positive: 671196, number of negative: 664163
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 33408
[LightGBM] [Info] Number of data points in the train set: 1335359, number of used features: 132
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 132 dense feature groups (168.10 MB) transferred to GPU in 0.195163 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502633 -> initscore=0.010534
[LightGBM] [Info] Start training from score 0.010534
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.64841	valid_1's auc: 0.533403
Early stopping, best iteration is:
[48]	training's auc: 0.646042	valid_1's auc: 0.534148


 80%|████████  | 4/5 [05:16<01:11, 71.09s/it]

[LightGBM] [Info] Number of positive: 671589, number of negative: 663770
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 33408
[LightGBM] [Info] Number of data points in the train set: 1335359, number of used features: 132
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 132 dense feature groups (168.10 MB) transferred to GPU in 0.209714 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502928 -> initscore=0.011711
[LightGBM] [Info] Start training from score 0.011711
Training until validation scores don't improve for 50 rounds
[50]	training's auc: 0.660963	valid_1's auc: 0.52496
[100]	training's auc: 0.705434	valid_1's auc: 0.524002
Early stopping, best iteration is:
[54]	training's auc: 0.665764	valid_1's auc: 0.526047


100%|██████████| 5/5 [06:13<00:00, 74.62s/it]


In [9]:
from sklearn.metrics import roc_auc_score
valid_pred=np.stack([model.predict(X_valid) for model in models]).T
valid_auc = roc_auc_score(y_valid, valid_pred)
print(valid_auc)

0.5506497969235709
